In [309]:
import pandas as pd
import torch
from torch.utils.data import DataLoader,Dataset
from torch import nn

## Model

In [310]:
class Classcifier(nn.Module):
    def __init__(self):
        super(Classcifier,self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(7,6),
            nn.ReLU(),
            nn.Linear(6,5),
            nn.ReLU(),
            nn.Linear(5,4),
            nn.ReLU(),
            nn.Linear(4,3),
            nn.ReLU(),
            nn.Linear(3,2),
        )
        self.softmax = nn.Softmax()
    def forward(self, x):
        x = self.layers(x)
        x = self.softmax(x)
        return x

## Dataset

In [311]:
class DatasetTantic(Dataset):
    def __init__(self,mode = "train"):
        if mode == "train":
            self.csv = pd.read_csv("./data/titanic/titanic_train_pro.csv")
            self.features = self.csv.loc[:,["Pclass","Sex","Age","SibSp","Parch","Fare","Embarked"]]
            self.targets = torch.nn.functional.one_hot(torch.tensor(self.csv.loc[:,"Survived"].values))
        elif mode == "test":
            self.csv = pd.read_csv("./data/titanic/titanic_train_pro.csv")

    def __len__(self):
        return len(self.csv)

    def __getitem__(self, idx):
        return torch.tensor(self.features.loc[idx,:].values).float() , self.targets[idx].float()

In [312]:
train_data = DatasetTantic()
train_dataloader = DataLoader(train_data,batch_size=64,shuffle=True)
model = Classcifier()

## loss func and optimier

In [313]:
lossFunc = torch.nn.CrossEntropyLoss()
lossFunc = lossFunc.cuda()
optimizer = torch.optim.SGD(model.parameters(),lr=0.05,momentum=0.9)

## Train loop

In [314]:
epochNum = 10000
device = "cuda"
for epoch in range(epochNum):
    lossTotal = 0
    model.train()
    model.cuda()
    for i,data in enumerate(train_dataloader):
        featrues, targets = data
        featrues = featrues.cuda()
        targets = targets.cuda()

        pres = model.forward(featrues)
        loss = lossFunc(pres,targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        lossTotal+= loss.item()
    print("lossTotal: {}".format(lossTotal))
torch.save(model.state_dict(),"./model/titanic_model.pt")


/tmp/ipykernel_73/1452074342.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


lossTotal: 12.655202865600586
lossTotal: 11.961817026138306
lossTotal: 11.788751542568207
lossTotal: 11.589807987213135
lossTotal: 11.275514960289001
lossTotal: 11.104258060455322
lossTotal: 11.013181149959564
lossTotal: 11.064688920974731
lossTotal: 11.029880464076996
lossTotal: 11.054188132286072
lossTotal: 11.089581906795502
lossTotal: 11.024233788251877
lossTotal: 11.12475061416626
lossTotal: 11.002059161663055
lossTotal: 11.019054472446442
lossTotal: 10.999373257160187
lossTotal: 10.997521698474884
lossTotal: 11.000320017337799
lossTotal: 10.955580413341522
lossTotal: 10.963543713092804
lossTotal: 11.068443953990936
lossTotal: 11.01469075679779
lossTotal: 10.915899753570557
lossTotal: 10.926722288131714
lossTotal: 10.946016788482666
lossTotal: 10.918609917163849
lossTotal: 10.920356035232544
lossTotal: 10.878590106964111
lossTotal: 10.995953142642975
lossTotal: 10.941039502620697
lossTotal: 10.951576799154282
lossTotal: 10.933503270149231
lossTotal: 10.883525133132935
lossTotal: 1